In [6]:
import quantumpropagator as qp
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interact,fixed #, interactive, fixed, interact_manual
import ipywidgets as widgets

import os
import sys
import glob
import pandas as pd
import numpy as np

from scipy import fftpack
%load_ext Cython
%matplotlib ipympl

#plt.rcParams.update({'font.size': 10})

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [2]:
fol = '/home/alessio/w-August-Run/'
outputFolder = '/home/alessio/w-August-Run/OUTPUTS'
subfolders2 = sorted([dir for dir in os.listdir(fol) if os.path.isdir(os.path.join(fol,dir))])
subfolders =  [ dir for dir in subfolders2 if dir not in ['HTML','csv','OUTPUTS'] ]
print(''.join(['{} -> {}\n'.format(a,b) for a,b in enumerate(subfolders)]))

0 -> b-UV-0.22_0000
1 -> m-IR-Polarized-INV_0000
2 -> m-IR-Polarized-LON_0000
3 -> m-IR-Polarized-NOR_0000
4 -> position-from1_0000
5 -> position-from2_0000
6 -> position-from3_0000
7 -> position-from4_0000
8 -> position-from5_0000
9 -> position-from6_0000
10 -> t-FC_excited_phi_0000
11 -> t-IR-to-excite-phi_0000
12 -> t-UV-0.22_excited_phi_0001
13 -> x-SubMask_from_IR_0000
14 -> x-SubMask_from_IR_INV_0000
15 -> z-from1_0000



In [3]:
%%cython
# -a
import numpy as np
import quantumpropagator as qp

cdef extern from "complex.h":
        double complex cexp(double complex)

# from libc.math cimport exp

# cdef extern from "<complex.h>" namespace "std":
#     double complex exp(double complex z)
#     float complex exp(float complex z)  # overload

def fft_artisanal(time,signal):
    dt = time[1] - time[0]
    nstep = time.size
    all_time = nstep * dt
    sx = -np.pi/dt
    dx = np.pi/dt
    dw = (2 * np.pi)/all_time
    freq = np.arange(0,dx,dw)
    freq_size = freq.size
    fft_array, freq = fft_c(time, signal, freq, dt, freq_size, nstep)
    return (fft_array, np.array(freq))

cdef fft_c(double [:] time, double [:] signal, double [:] freq, int dt, int freq_size, int nstep):
    cdef:
        int k,j
        double complex I = -1j
        
    fft_array = np.zeros(freq_size, dtype=complex)
    
    for k in range(freq_size):
        for j in range(nstep):

            fft_array[k] = fft_array[k] + cexp(I * freq[k] * time[j]) * signal[j]

    return(fft_array, freq)

In [4]:
# define all labels
nstates = 8
permanents = []
transitions = []

for lab1 in ['x','y','z']:
    for lab2 in range(nstates):
        permanents.append('perm_{}_{}'.format(lab1,lab2))
        for lab3 in range(lab2+1,nstates):
            transitions.append('trans_{}_{}_{}'.format(lab1,lab2,lab3))
            

print(permanents + transitions)

['perm_x_0', 'perm_x_1', 'perm_x_2', 'perm_x_3', 'perm_x_4', 'perm_x_5', 'perm_x_6', 'perm_x_7', 'perm_y_0', 'perm_y_1', 'perm_y_2', 'perm_y_3', 'perm_y_4', 'perm_y_5', 'perm_y_6', 'perm_y_7', 'perm_z_0', 'perm_z_1', 'perm_z_2', 'perm_z_3', 'perm_z_4', 'perm_z_5', 'perm_z_6', 'perm_z_7', 'trans_x_0_1', 'trans_x_0_2', 'trans_x_0_3', 'trans_x_0_4', 'trans_x_0_5', 'trans_x_0_6', 'trans_x_0_7', 'trans_x_1_2', 'trans_x_1_3', 'trans_x_1_4', 'trans_x_1_5', 'trans_x_1_6', 'trans_x_1_7', 'trans_x_2_3', 'trans_x_2_4', 'trans_x_2_5', 'trans_x_2_6', 'trans_x_2_7', 'trans_x_3_4', 'trans_x_3_5', 'trans_x_3_6', 'trans_x_3_7', 'trans_x_4_5', 'trans_x_4_6', 'trans_x_4_7', 'trans_x_5_6', 'trans_x_5_7', 'trans_x_6_7', 'trans_y_0_1', 'trans_y_0_2', 'trans_y_0_3', 'trans_y_0_4', 'trans_y_0_5', 'trans_y_0_6', 'trans_y_0_7', 'trans_y_1_2', 'trans_y_1_3', 'trans_y_1_4', 'trans_y_1_5', 'trans_y_1_6', 'trans_y_1_7', 'trans_y_2_3', 'trans_y_2_4', 'trans_y_2_5', 'trans_y_2_6', 'trans_y_2_7', 'trans_y_3_4', 'trans

In [20]:
plt.close('all')

def reorder_dataframe_last_first(df):
    '''used to reorder dataframe'''
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    return df[cols]

def process_folder(this_folder,fol,outputFolder):
    project_folder = os.path.join(fol,this_folder)
    output_dipo = os.path.join(project_folder, 'Output_Dipole')
    all_labels = ['fs_2','dipx','dipy','dipz'] +  permanents + transitions
    df_dipo2 = pd.read_csv(output_dipo, delim_whitespace=True, names=all_labels)
    
    fran_df_dipole = pd.DataFrame()
    fran_df_freq = pd.DataFrame()
    
    for cart in ['x','y','z']:
        full_list_label = permanents + transitions
        full_list_this_cart = [x for x in full_list_label if cart in x]

        # setting figure
        fig, [ax0,ax1,ax2] = plt.subplots(3,1,figsize = (12,8))

        # first panel is just total signal
        timefs = df_dipo2['fs_2']
        time = qp.fromFsToAu(timefs)
        tot_signal = df_dipo2['dip{}'.format(cart)]
        ax0.plot(timefs,tot_signal)
        

        #second panel is big components
        threshold = np.linalg.norm(tot_signal)*0.05 # 5% of the norm
        big_components = [ x for x in full_list_this_cart if np.linalg.norm(df_dipo2[x]) > threshold ] # the one which contributes more than 5% to the norm
        print(big_components)

        for this_lab in big_components:
            ax1.plot(timefs, df_dipo2[this_lab], label=this_lab)
            
            
            
    # THIS PART FOR DFT
            dt = time[1]-time[0]

            t = time
            x = df_dipo2[this_lab]

            fft_array, freq = fft_artisanal(np.array(t),np.array(x))
#             print(fft_array.shape, freq.shape)

            ev_freq = qp.fromHartoEv(freq)
            abs_fft_array = np.abs(fft_array)
            ax2.plot(ev_freq, abs_fft_array, label=this_lab)

            # add the things into Francoise df
#             print(df_dipo2[this_lab].shape,fft_array.shape)
            fran_df_dipole[this_lab] = df_dipo2[this_lab]
            fran_df_freq['DFT-{}'.format(this_lab)] = abs_fft_array
        

        
    # #  THOSE LINES TO DEBUG WITH GNUPLOT
    # #     degub_fodler = '/home/alessio/Desktop/'

    # #     np.savetxt(os.path.join(degub_fodler,'file1'),a)
    # #     np.savetxt(os.path.join(degub_fodler,'file2'),b)

        
        ax1.legend(ncol=5)
        ax1.set_xlabel('fs')
        ax2.legend(ncol=5)
        ax2.set_xlabel('eV')
        ax0.set_title('Dipole {}'.format(cart))
        fig.tight_layout()
    # this needs to be ONCE in the df of Francoise
    fran_df_dipole['time fs'] = timefs
    fran_df_freq['DFT-Frequencies eV'] = freq
    fran_df_dipole = reorder_dataframe_last_first(fran_df_dipole)
    fran_df_freq = reorder_dataframe_last_first(fran_df_freq)
    
    fran_df_dipole.to_csv(os.path.join(outputFolder,'{}_dipoles.csv'.format(this_folder)))
    fran_df_freq.to_csv(os.path.join(outputFolder,'{}_dipoles_DFT.csv'.format(this_folder)))
    
    
    return(df_dipo2['dip{}'.format('z')],timefs,fran_df_dipole,fran_df_freq)
    
dipz,time,fran_df_dipole,fran_df_freq = process_folder(subfolders[0],fol,outputFolder)

A Jupyter Widget

['trans_x_0_1', 'trans_x_1_2', 'trans_x_1_3', 'trans_x_1_4']


A Jupyter Widget

['trans_y_0_1', 'trans_y_1_2', 'trans_y_1_3', 'trans_y_1_4', 'trans_y_1_5', 'trans_y_2_3', 'trans_y_6_7']


A Jupyter Widget

['perm_z_0', 'perm_z_1', 'trans_z_1_4']


# Try to clean signal part

In [118]:
def adjust_signal(sig,ext):
    sig_size = sig.size
    array_value = np.zeros(ext)
    sig_array = np.array(dipz)
    flipped = np.flip(sig_array,axis=0)
    
    output_signal = np.concatenate((array_value,flipped,sig_array,array_value))
    return(output_signal)

def adjust_time(time,full_length):
    time_size = time.size
    time_final_value = time[time_size-1]
    total_time = np.linspace(-time_final_value,time_final_value,full_length)
    return(total_time)
    


sig_ext = adjust_signal(dipz,1000)
time_ext = adjust_time(time,10002)

In [119]:
#plt.close('all')

import scipy
fig, [ax0, ax1] = plt.subplots(2,1,figsize = (12,8))

a = scipy.signal.windows.gaussian(10002,1400)
z = adjust_signal(dipz,1000)

newsignal = z*a

ax0.plot(time_ext,a)
ax0.plot(time_ext,newsignal)

fft_array, freq = fft_artisanal(time_ext,newsignal)

ax1.plot(freq, np.abs(fft_array))

fig.tight_layout();

A Jupyter Widget